<a href="https://colab.research.google.com/github/mostafa-ja/mal_adv3/blob/main/4_adverserial_attacks_drebin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

download_links = ['https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_0.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_1.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_2.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y0.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y1.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y2.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_DNN_drebin_best.pth',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/data/adverserial_attacks_functions.py',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_AT_rFGSM_weightedLoss.pth',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_AT_rFGSM.pth'
]

In [2]:
import gdown
output_filepath = '/content/'
for link in download_links:
  gdown.download(link, output_filepath)


Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_0.npz
To: /content/sparse_matrix_0.npz
100%|██████████| 461k/461k [00:00<00:00, 5.40MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_1.npz
To: /content/sparse_matrix_1.npz
100%|██████████| 148k/148k [00:00<00:00, 1.68MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_2.npz
To: /content/sparse_matrix_2.npz
100%|██████████| 150k/150k [00:00<00:00, 2.01MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y0.npz
To: /content/sparse_matrix_y0.npz
100%|██████████| 5.79k/5.79k [00:00<00:00, 10.9MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y1.npz
To: /content/sparse_matrix_y1.npz
100%|██████████| 2.64k/2.64k [00:00<00:00, 4.20MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y2

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,balanced_accuracy_score
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import pickle
import random
import time

from adverserial_attacks_functions import *

torch.manual_seed(0)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:

# Load dataset
X_train = sparse.load_npz("/content/sparse_matrix_0.npz").toarray()
X_val = sparse.load_npz("/content/sparse_matrix_1.npz").toarray()
X_test = sparse.load_npz("/content/sparse_matrix_2.npz").toarray()

X_train = torch.tensor(X_train, dtype=torch.int8)
X_val = torch.tensor(X_val, dtype=torch.int8)
X_test = torch.tensor(X_test, dtype=torch.int8)


y_train = sparse.load_npz("/content/sparse_matrix_y0.npz").toarray().reshape((-1, 1))
y_val = sparse.load_npz("/content/sparse_matrix_y1.npz").toarray().reshape((-1, 1))
y_test = sparse.load_npz("/content/sparse_matrix_y2.npz").toarray().reshape((-1, 1))

y_train = torch.tensor(y_train, dtype=torch.int8)
y_val = torch.tensor(y_val, dtype=torch.int8)
y_test = torch.tensor(y_test, dtype=torch.int8)


print("Shapes:")
print("x_train:", X_train.shape)
print("x_val:", X_val.shape)
print("x_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_val:", y_val.shape)
print("y_test:", y_test.shape)

Shapes:
x_train: torch.Size([28683, 10000])
x_val: torch.Size([9562, 10000])
x_test: torch.Size([9562, 10000])
y_train: torch.Size([28683, 1])
y_val: torch.Size([9562, 1])
y_test: torch.Size([9562, 1])


In [5]:
# number of benigns and malicious sample in training dataset
n_ben = (y_train.squeeze()== 0).sum().item()
n_mal = (y_train.squeeze()== 1).sum().item()
print('the proportion of malwares : ', n_mal/(n_mal+n_ben))

# Combine features and labels into datasets
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

# Define the DataLoader for training, validation, and test sets
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Clear unnecessary variables
del train_dataset, val_dataset, test_dataset, y_train, y_val, y_test


the proportion of malwares :  0.11386535578565701


In [6]:
class MalwareDetectionModel(nn.Module):
    def __init__(self, input_size=10000, hidden_1_size=200, hidden_2_size=200, num_labels=2, dropout_prob=0.6):
        super(MalwareDetectionModel, self).__init__()

        self.input_size = input_size
        self.hidden_1_size = hidden_1_size
        self.hidden_2_size = hidden_2_size
        self.num_labels = num_labels
        self.dropout_prob = dropout_prob

        self.fc1 = nn.Linear(input_size, hidden_1_size)
        self.relu1 = nn.ReLU()
        #self.dropout1 = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(hidden_1_size, hidden_2_size)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_prob)
        self.fc3 = nn.Linear(hidden_2_size, num_labels)
        #self.log_softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        #x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        #x = self.log_softmax(x)
        return x


In [7]:
model_DNN = MalwareDetectionModel().to(device)
# Load model parameters
model_DNN.load_state_dict(torch.load('model_DNN_drebin_best.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [ ]:
def model_evaluation(model, test_loader, device):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    true_labels = []

    with torch.no_grad():
        for X_batch, labels_batch in test_loader:
            X_batch, labels_batch = X_batch.to(torch.float32).to(device), labels_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.topk(outputs, k=1)
            predictions.extend(predicted.tolist())
            true_labels.extend(labels_batch.tolist())

    # Convert predictions and true labels to numpy arrays
    predictions = np.array(predictions)
    true_labels = np.array(true_labels)

    # Calculate and print test accuracy
    accuracy = accuracy_score(true_labels, predictions)
    balanced_acc = balanced_accuracy_score(true_labels, predictions)
    print(f'Test Accuracy: {accuracy:.4f}')
    print(f'Test balanced Accuracy: {balanced_acc:.4f}')

    # Calculate and print precision, recall, and F1-score
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)

    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1-score: {f1:.4f}')

    # Calculate and print true positives, true negatives, false positives, and false negatives
    TP = ((predictions == 1) & (true_labels == 1)).sum()
    TN = ((predictions == 0) & (true_labels == 0)).sum()
    FP = ((predictions == 1) & (true_labels == 0)).sum()
    FN = ((predictions == 0) & (true_labels == 1)).sum()

    print(f'True Positives (TP): {TP}')
    print(f'True Negatives (TN): {TN}')
    print(f'False Positives (FP): {FP}')
    print(f'False Negatives (FN): {FN}')

    # Calculate and print False Negative Rate (FNR) and False Positive Rate (FPR)
    FNR = (FN / (FN + TP)) * 100
    FPR = (FP / (FP + TN)) * 100

    print(f'False Negative Rate (FNR): {FNR:.4f}')
    print(f'False Positive Rate (FPR): {FPR:.4f}')

In [ ]:
def adjust_learning_rate(optimizer, epoch, lr_step=(25,35,45), lr_decay_ratio=0.2):
    """Adjust the learning rate based on the epoch number."""
    if epoch == 0:
        optimizer.param_groups[0]['lr'] /= 8
    elif epoch in [1, 2, 3]:  # in step five , we finish warm up ,and start normal learning rate
        optimizer.param_groups[0]['lr'] *= 2
    if epoch in lr_step: # in these steps , we are geting close to optimal point so we need to have shorter step
        optimizer.param_groups[0]['lr'] *= lr_decay_ratio
    return optimizer


In [ ]:
def adversarial_training(model, train_loader, val_loader, attack, adv_epochs=50, lr=0.001, weight_decay=0., device=device, verbose=True, **kwargs):

    # Assuming positive class (malware) is label 1
    class_weights = torch.tensor([0.11, 0.89]).to(device)  # Adjust the weights based on the class distribution, higher weight for positive class

    # Define Loss Function and Optimizer
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    total_time = 0.
    nbatches = len(train_loader)
    best_acc_val = 0.
    acc_val_adv_be = 0.
    best_epoch = 0

    for epoch in range(adv_epochs):
        epoch_losses = []
        epoch_accuracies = []
        optimizer = adjust_learning_rate(optimizer, epoch)

        for idx_batch, (x_batch, y_batch) in enumerate(train_loader):
            x_batch, y_batch = x_batch.to(torch.float32).to(device), y_batch.to(device)
            batch_size = x_batch.shape[0]

            # Separate malicious and benign samples
            mal_x_batch, ben_x_batch = x_batch[y_batch.squeeze() == 1], x_batch[y_batch.squeeze() == 0]
            mal_y_batch, ben_y_batch = y_batch[y_batch.squeeze() == 1], y_batch[y_batch.squeeze() == 0]

            # Generate adversarial examples
            model.eval()
            pertb_mal_x = attack(mal_x_batch, mal_y_batch, model, **kwargs)
            x_batch = torch.cat([ben_x_batch, pertb_mal_x], dim=0)
            y_batch = torch.cat([ben_y_batch, mal_y_batch])
            model.train()

            # Forward pass and backward pass
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss_train = criterion(outputs, y_batch.view(-1).long())
            loss_train.backward()
            optimizer.step()

            # Calculate metrics
            epoch_losses.append(loss_train.item())
            predicted = outputs.argmax(1).unsqueeze(1)
            acc_train = (predicted == y_batch).sum().item() / len(y_batch)
            epoch_accuracies.append(acc_train)

            # Print batch level information
            #if verbose:
                #print(f'Mini batch: {idx_batch + 1}/{nbatches} | Epoch: {epoch + 1}/{adv_epochs} | Batch Loss: {loss_train.item():.4f} | Batch Accuracy: {acc_train * 100:.2f}%')

        # Calculate epoch level metrics
        mean_loss = np.mean(epoch_losses)
        mean_accuracy = np.mean(epoch_accuracies) * 100

        # Print epoch level information
        if verbose:
            print(f'Epoch: {epoch+1}/{adv_epochs} | Training loss (epoch level): {mean_loss:.4f} | Train accuracy: {mean_accuracy:.2f}%')

        # Evaluation on validation set
        model.eval()
        avg_acc_ad_val = []
        avg_acc_val = []
        for x_val, y_val in val_loader:
            x_val, y_val = x_val.to(torch.float32).to(device), y_val.to(device)
            outputs = model(x_val)
            predicted = outputs.argmax(1).unsqueeze(1)
            acc_val = (predicted == y_val).sum().item() / len(y_val)
            avg_acc_val.append(acc_val)

            # Generate adversarial examples for validation set
            mal_x_batch, mal_y_batch = x_val[y_val.squeeze() == 1], y_val[y_val.squeeze() == 1]
            pertb_mal_x = attack(mal_x_batch, mal_y_batch, model, **kwargs)
            outputs = model(pertb_mal_x)
            y_pred = outputs.argmax(1).unsqueeze(1)

            acc_ad_val = (y_pred == 1.).sum().item() / len(y_pred)
            avg_acc_ad_val.append(acc_ad_val)

        # Calculate validation accuracy
        assert len(avg_acc_ad_val) > 0
        acc_all = (np.mean(avg_acc_val) + np.mean(avg_acc_ad_val)) / 2.

        # Update best validation accuracy
        if acc_all >= best_acc_val:
            best_acc_val = acc_all
            acc_val_adv_be = np.mean(avg_acc_ad_val)
            best_epoch = epoch + 1
            torch.save(model.state_dict(), 'best_AT_model.pth')

        # Print validation results
        if verbose:
            print(f"\tVal accuracy(without attack) {np.mean(avg_acc_val) * 100:.4}% and accuracy(with attack) {np.mean(avg_acc_ad_val) * 100:.4}% under attack and overall accuracy {acc_all * 100:.4}%.")
            print(f"\tModel select at epoch {best_epoch} with validation accuracy {best_acc_val * 100:.4}% and accuracy {acc_val_adv_be * 100:.4}% under attack.")


In [ ]:
# AT-rFGSM: Adversarial Taraining based on rFGSM attack
model_AT = MalwareDetectionModel().to(device)

attack_param = {"k":50, "epsilon":0.02, 'random':True, "is_sample":False, 'is_report_loss_diff':False}
adversarial_training(model_AT, train_loader, val_loader, adv_epochs=50, attack=dfgsm_k, **attack_param)

Training loss (epoch level): 0.4688 | Train accuracy: 84.47%
	Val accuracy(without attack) 96.55% and accuracy(with attack) 71.52% under attack and overall accuracy 84.04%.
	Model select at epoch 1 with validation accuracy 84.04% and accuracy 71.52% under attack.
Training loss (epoch level): 0.1100 | Train accuracy: 96.80%
	Val accuracy(without attack) 98.09% and accuracy(with attack) 85.61% under attack and overall accuracy 91.85%.
	Model select at epoch 2 with validation accuracy 91.85% and accuracy 85.61% under attack.
Training loss (epoch level): 0.0643 | Train accuracy: 98.12%
	Val accuracy(without attack) 98.39% and accuracy(with attack) 89.02% under attack and overall accuracy 93.7%.
	Model select at epoch 3 with validation accuracy 93.7% and accuracy 89.02% under attack.
Training loss (epoch level): 0.0489 | Train accuracy: 98.43%
	Val accuracy(without attack) 98.78% and accuracy(with attack) 88.53% under attack and overall accuracy 93.66%.
	Model select at epoch 3 with validat

In [ ]:
# with weighted loss
# AT-rFGSM: Adversarial Taraining based on rFGSM attack
model_AT2 = MalwareDetectionModel().to(device)

attack_param = {"k":50, "epsilon":0.02, 'random':True, "is_sample":False, 'is_report_loss_diff':False}
adversarial_training(model_AT2, train_loader, val_loader, adv_epochs=50, attack=dfgsm_k, **attack_param)

Epoch: 1/50 | Training loss (epoch level): 0.5498 | Train accuracy: 80.06%
	Val accuracy(without attack) 90.37% and accuracy(with attack) 93.56% under attack and overall accuracy 91.96%.
	Model select at epoch 1 with validation accuracy 91.96% and accuracy 93.56% under attack.
Epoch: 2/50 | Training loss (epoch level): 0.2260 | Train accuracy: 94.87%
	Val accuracy(without attack) 95.99% and accuracy(with attack) 93.56% under attack and overall accuracy 94.77%.
	Model select at epoch 2 with validation accuracy 94.77% and accuracy 93.56% under attack.
Epoch: 3/50 | Training loss (epoch level): 0.1326 | Train accuracy: 96.29%
	Val accuracy(without attack) 95.99% and accuracy(with attack) 95.85% under attack and overall accuracy 95.92%.
	Model select at epoch 3 with validation accuracy 95.92% and accuracy 95.85% under attack.
Epoch: 4/50 | Training loss (epoch level): 0.1006 | Train accuracy: 96.87%
	Val accuracy(without attack) 96.51% and accuracy(with attack) 96.57% under attack and over

In [8]:
# Create an instance of your model
model_AT_rFGSM = MalwareDetectionModel().to(device)

# Load model parameters
model_AT_rFGSM.load_state_dict(torch.load('model_AT_rFGSM.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [9]:
# Create an instance of your model
model_AT_rFGSM_weightedLoss = MalwareDetectionModel().to(device)

# Load model parameters
model_AT_rFGSM_weightedLoss.load_state_dict(torch.load('model_AT_rFGSM_weightedLoss.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [ ]:
model_evaluation(model_DNN, test_loader, device)

Test Accuracy: 0.9912
Test balanced Accuracy: 0.9785
Precision: 0.9637
Recall: 0.9619
F1-score: 0.9628
True Positives (TP): 1087
True Negatives (TN): 8391
False Positives (FP): 41
False Negatives (FN): 43
False Negative Rate (FNR): 3.8053
False Positive Rate (FPR): 0.4862


In [ ]:
model_evaluation(model_AT_rFGSM,test_loader,device)

Test Accuracy: 0.9886
Test balanced Accuracy: 0.9663
Precision: 0.9654
Recall: 0.9372
F1-score: 0.9511
True Positives (TP): 1059
True Negatives (TN): 8394
False Positives (FP): 38
False Negatives (FN): 71
False Negative Rate (FNR): 6.2832
False Positive Rate (FPR): 0.4507


In [ ]:
model_evaluation(model_AT_rFGSM_weightedLoss,test_loader,device)

Test Accuracy: 0.9861
Test balanced Accuracy: 0.9745
Precision: 0.9257
Recall: 0.9593
F1-score: 0.9422
True Positives (TP): 1084
True Negatives (TN): 8345
False Positives (FP): 87
False Negatives (FN): 46
False Negative Rate (FNR): 4.0708
False Positive Rate (FPR): 1.0318


In [100]:
def adv_predict(test_loader, model, attack, device, **kwargs):

    if attack == mimicry:
      # Pre-select benign samples
      benign_samples = []
      for x_batch, y_batch in test_loader:
        benign_samples.append(x_batch[y_batch.squeeze() == 0])

      ben_x = torch.cat(benign_samples, dim=0).to(device)
      del benign_samples

    model.eval()
    avg_acc_ad_test = []
    avg_acc_test = []
    with torch.no_grad():
        for x_test, y_test in test_loader:
            x_test, y_test = x_test.to(torch.float32).to(device), y_test.to(device)
            #outputs = model(x_test)
            #predicted = outputs.argmax(1).unsqueeze(1)
            #acc_test = (predicted == y_test).sum().item() / len(y_test)
            #avg_acc_test.append(acc_test)

            mal_x_batch, mal_y_batch = x_test[y_test.squeeze() == 1], y_test[y_test.squeeze() == 1]

            outputs = model(mal_x_batch)
            predicted = outputs.argmax(1).unsqueeze(1)
            acc_test = (predicted == mal_y_batch).sum().item() / len(mal_y_batch)
            avg_acc_test.append(acc_test)

            # Generate adversarial examples for test set
            if attack == mimicry:
                pertb_mal_x = mimicry(ben_x, mal_x_batch, model, **kwargs)
            else :
                with torch.enable_grad():
                    pertb_mal_x = attack(mal_x_batch, mal_y_batch, model, **kwargs)

            outputs = model(pertb_mal_x)
            y_pred = outputs.argmax(1).unsqueeze(1)
            print(y_pred.shape)
            acc_ad_test = (y_pred == 1.).sum().item() / len(y_pred)
            avg_acc_ad_test.append(acc_ad_test)

    # Calculate test accuracy
    print(f"Accuracy of just malwares (without attack): {np.mean(avg_acc_test) * 100:.4}% | Under attack: {np.mean(avg_acc_ad_test) * 100:.4}%.")
    if attack == mimicry:
      del ben_x


In [115]:
def adv_predict(test_loader, model, attack, device, **kwargs):

    if attack == mimicry:
      # Pre-select benign samples
      benign_samples = []
      for x_batch, y_batch in test_loader:
        benign_samples.append(x_batch[y_batch.squeeze() == 0])

      ben_x = torch.cat(benign_samples, dim=0).to(device)
      del benign_samples

    model.eval()
    n_samples = 0
    cor_test = 0
    cor_ad_test = 0

    with torch.no_grad():
        for x_test, y_test in test_loader:
            x_test, y_test = x_test.to(torch.float32).to(device), y_test.to(device)
            #outputs = model(x_test)
            #predicted = outputs.argmax(1).unsqueeze(1)
            #acc_test = (predicted == y_test).sum().item() / len(y_test)
            #avg_acc_test.append(acc_test)

            mal_x_batch, mal_y_batch = x_test[y_test.squeeze() == 1], y_test[y_test.squeeze() == 1]
            n_samples += len(mal_y_batch)

            outputs = model(mal_x_batch)
            predicted = outputs.argmax(1)
            cor_test += (predicted == 1).sum().item()

            # Generate adversarial examples for test set
            if attack == mimicry:
                pertb_mal_x = mimicry(ben_x, mal_x_batch, model, **kwargs)
            else :
                with torch.enable_grad():
                    pertb_mal_x = attack(mal_x_batch, mal_y_batch, model, **kwargs)

            outputs = model(pertb_mal_x)
            y_pred = outputs.argmax(1)
            cor_ad_test += (y_pred == 1).sum().item()

    # Calculate test accuracy
    print(f"Accuracy of just malwares (without attack): {(cor_test / n_samples) * 100:.4}% | Under attack: {(cor_ad_test / n_samples) * 100:.4}%.")
    if attack == mimicry:
      del ben_x


In [ ]:
# rFGSM
attack_params = {"k":50, "epsilon":0.02, 'random':True, 'is_report_loss_diff':False, 'is_sample':False}
adv_predict(test_loader, model_DNN, dfgsm_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, dfgsm_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, dfgsm_k, device, **attack_params)


Malwares Dtection accuracy (without attack): 96.01% | Under attack: 0.0%.
Malwares Dtection accuracy (without attack): 93.77% | Under attack: 93.2%.
Malwares Dtection accuracy (without attack): 95.81% | Under attack: 94.8%.


In [ ]:
# rFGSM
attack_params = {"k":100, "epsilon":0.02, 'random':True, 'is_report_loss_diff':False, 'is_sample':False}
adv_predict(test_loader, model_DNN, dfgsm_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, dfgsm_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, dfgsm_k, device, **attack_params)

Malwares Dtection accuracy (without attack): 96.01% | Under attack: 0.0%.
Malwares Dtection accuracy (without attack): 93.77% | Under attack: 93.52%.
Malwares Dtection accuracy (without attack): 95.81% | Under attack: 94.81%.


In [ ]:
adv_predict(test_loader, model_AT_rFGSM, attack=pgd, is_report_loss_diff=False, device=device)

Adversarial accuracy (without attack): 99.36% | Under attack: 90.83%.


In [ ]:
# Groose
attack_params = {"k":100, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, grosse_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, grosse_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, grosse_k, device, **attack_params)

Malwares Dtection accuracy (without attack): 96.01% | Under attack: 0.0%.
Malwares Dtection accuracy (without attack): 93.77% | Under attack: 86.91%.
Malwares Dtection accuracy (without attack): 95.81% | Under attack: 45.7%.


In [ ]:
# BCA
attack_params = {"k":100, 'is_report_loss_diff':False, 'use_sample':False}
adv_predict(test_loader, model_DNN, bca_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, bca_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, bca_k, device, **attack_params)

Malwares Dtection accuracy (without attack): 96.01% | Under attack: 0.0%.
Malwares Dtection accuracy (without attack): 93.77% | Under attack: 86.91%.
Malwares Dtection accuracy (without attack): 95.81% | Under attack: 47.22%.


In [ ]:
# BGA
attack_params = {"k":100, 'is_report_loss_diff':False, 'use_sample':False}
adv_predict(test_loader, model_DNN, bga_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, bga_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, bga_k, device, **attack_params)

Malwares Dtection accuracy (without attack): 96.01% | Under attack: 0.0%.
Malwares Dtection accuracy (without attack): 93.77% | Under attack: 92.66%.
Malwares Dtection accuracy (without attack): 95.81% | Under attack: 89.43%.


In [ ]:
# rFGSM
attack_params = {"k":100, "epsilon":0.02, 'random':True, 'is_report_loss_diff':False, 'is_sample':False}
adv_predict(test_loader, model_DNN, dfgsm_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, dfgsm_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, dfgsm_k, device, **attack_params)

Malwares Dtection accuracy (without attack): 96.01% | Under attack: 0.0%.
Malwares Dtection accuracy (without attack): 93.77% | Under attack: 93.31%.
Malwares Dtection accuracy (without attack): 95.81% | Under attack: 94.82%.


In [ ]:
# PGD-l1
attack_params = {"k":500, "step_length":1., 'norm':'l1', 'random':False, 'is_report_loss_diff':False, 'is_sample':False}
adv_predict(test_loader, model_DNN, pgd, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, pgd, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, pgd, device, **attack_params)

Malwares Dtection accuracy (without attack): 96.01% | Under attack: 0.0%.
Malwares Dtection accuracy (without attack): 93.77% | Under attack: 86.91%.
Malwares Dtection accuracy (without attack): 95.81% | Under attack: 46.9%.


In [ ]:
# PGD-l2
attack_params = {"k":200, "step_length":0.05, 'norm':'l2', 'random':False, 'is_report_loss_diff':False, 'is_sample':False}
adv_predict(test_loader, model_DNN, pgd, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, pgd, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, pgd, device, **attack_params)

Malwares Dtection accuracy (without attack): 96.01% | Under attack: 31.76%.
Malwares Dtection accuracy (without attack): 93.77% | Under attack: 93.77%.
Malwares Dtection accuracy (without attack): 95.81% | Under attack: 95.81%.


In [ ]:
# PGD-linf
attack_params = {"k":500, "step_length":0.002, 'norm':'linf', 'random':False, 'is_report_loss_diff':False, 'is_sample':False}
adv_predict(test_loader, model_DNN, pgd, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, pgd, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, pgd, device, **attack_params)

Malwares Dtection accuracy (without attack): 96.01% | Under attack: 0.0%.
Malwares Dtection accuracy (without attack): 93.77% | Under attack: 89.12%.
Malwares Dtection accuracy (without attack): 95.81% | Under attack: 72.61%.


In [126]:
# Mimicry×1
attack_params = {"trials":1, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, mimicry, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 78.94%.
Accuracy of just malwares (without attack): 93.72% | Under attack: 93.63%.
Accuracy of just malwares (without attack): 95.93% | Under attack: 96.37%.


In [127]:
# Mimicry×10
attack_params = {"trials":10, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, mimicry, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 34.87%.
Accuracy of just malwares (without attack): 93.72% | Under attack: 92.65%.
Accuracy of just malwares (without attack): 95.93% | Under attack: 94.42%.


In [128]:
# Mimicry×30
attack_params = {"trials":30, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, mimicry, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 30.18%.
Accuracy of just malwares (without attack): 93.72% | Under attack: 92.39%.
Accuracy of just malwares (without attack): 95.93% | Under attack: 93.81%.


In [130]:
# Mimicry×100
attack_params = {"trials":100, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, mimicry, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 30.18%.
Accuracy of just malwares (without attack): 93.72% | Under attack: 92.3%.
Accuracy of just malwares (without attack): 95.93% | Under attack: 93.54%.


In [133]:
# Mimicry×1000
attack_params = {"trials":1000, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, mimicry, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 16.73%.
Accuracy of just malwares (without attack): 93.72% | Under attack: 91.77%.
Accuracy of just malwares (without attack): 95.93% | Under attack: 91.06%.


In [134]:
def mimicry(ben_x, malicious_samples, model, trials=30, seed=230, is_report_loss_diff=False):
    """
    Perform a mimicry attack.

    Args:
    - ben_x (torch.Tensor): Benign samples tensor.
    - malicious_samples (torch.Tensor): Malicious samples tensor.
    - model (torch.nn.Module): PyTorch model used for the attack.
    - trials (int): Number of trials for the attack.
    - seed (int): Random seed for reproducibility.
    - is_report_loss_diff (bool): Flag to indicate whether to report attack effectiveness.

    Returns:
    - adv_x (torch.Tensor): Adversarial examples tensor.
    """
    model.eval()
    seed += 1
    torch.manual_seed(seed)

    # Ensure trials do not exceed the length of ben_x
    trials = min(trials, len(ben_x))

    # Get the number of malicious samples
    n_samples = len(malicious_samples)

    if n_samples > 0:
        # Expand dimensions for efficient broadcasting
        malicious_samples_expanded = malicious_samples.unsqueeze(1).expand(-1, trials, -1)

        # Generate random indices for sampling from ben_x
        indices = torch.randperm(len(ben_x), device=ben_x.device)[:trials]
        trial_vectors_expanded = ben_x[indices].unsqueeze(0)

        # Perform the mimic attack
        pertbx = torch.clamp(malicious_samples_expanded + trial_vectors_expanded, min=0., max=1.)

        # Compute the loss and check if adversarial examples are successful
        loss, done = get_loss(pertbx.view(-1, pertbx.shape[-1]), torch.ones(n_samples * trials, 1, device=ben_x.device), model)

        # Add maximum loss to successful attacks to differentiate
        max_v = loss.max()
        loss[done] += max_v

        # Reshape the loss and done tensors
        loss = loss.view(n_samples, trials)
        done = done.view(n_samples, trials)

        # Report attack effectiveness if required
        if is_report_loss_diff:
            n_done = torch.any(done, dim=-1).sum()
            print(f"Mimicry*{trials}: Attack effectiveness {n_done / n_samples * 100:.3f}%.")

        # Get the index of the maximum loss for each sample
        _, indices = loss.max(dim=-1)
        adv_x = pertbx[torch.arange(n_samples), indices]

        del pertbx, loss, done, malicious_samples_expanded, trial_vectors_expanded

        return adv_x
    else:
        print("No malicious samples found.")
        return None

In [91]:
def mimic_attack_effectiveness_optimized(test_loader, model, seed, trials=1000, device="cuda:0"):
  """
  Calculates the effectiveness of the mimic attack on the given model.

  Args:
      test_loader: A PyTorch dataloader containing the test data.
      model: The PyTorch model to be attacked.
      seed: The random seed for reproducibility.
      trials: The number of random samples to use from the benign class (default: 1000).
      device: The device to use for computations (default: "cuda:0" if available, otherwise "cpu").

  Returns:
      The effectiveness of the mimic attack as a percentage (float).
  """

  torch.manual_seed(seed)
  model.eval()

  # Initialize counters
  successful_attacks = 0
  total_malicious_samples = 0

  # Pre-select benign samples for efficiency
  benign_samples = []
  for x_batch, y_batch in test_loader:
    benign_samples.append(x_batch[y_batch.squeeze() == 0])

  ben_x = torch.cat(benign_samples, dim=0).to(device)

  # Clear unnecessary variables
  del benign_samples

  trials = min(trials, len(ben_x))


  for x_batch, y_batch in test_loader:
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
    malicious_samples = x_batch[y_batch.squeeze() == 1]

    if len(malicious_samples) > 0:
      # Expand dimensions for efficient broadcasting
      malicious_samples_expanded = malicious_samples.unsqueeze(1).expand(-1, trials, -1)

      # Generate random indices outside the loop
      #seed += 1
      torch.manual_seed(seed)
      indices = torch.randperm(len(ben_x), device=device)[:trials]
      trial_vectors_expanded = ben_x[indices].unsqueeze(0)

      # Perform the mimic attack and update counters
      modified_x = torch.clamp(malicious_samples_expanded + trial_vectors_expanded, min=0., max=1.)
      _, done = get_loss(modified_x.view(-1, modified_x.shape[-1]), torch.ones(trials * malicious_samples.shape[0], 1, device=device), model)
      successful_attacks += (done.view(malicious_samples.shape[0], trials).sum(dim=1) > 0).sum().item()
      total_malicious_samples += malicious_samples.shape[0]

  # Calculate and print attack effectiveness
  attack_effectiveness = (successful_attacks / total_malicious_samples) * 100 if total_malicious_samples > 0 else 0
  print(f"Mimic attack effectiveness: {attack_effectiveness:.3f}%.")

  #return attack_effectiveness  # Added return statement for clarity


In [131]:
mimic_attack_effectiveness_optimized(test_loader, model_DNN , seed=230, trials=1000, device=device)

Mimic attack effectiveness: 83.274%.


In [132]:
# Mimicry×100
attack_params = {"trials":1000, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 16.73%.


In [68]:
for x_test, y_test in test_loader:
  x_test, y_test = x_test.to(torch.float32).to(device), y_test.to(device)
  malicious_samples, mal_y_batch = x_test[y_test.squeeze() == 1], y_test[y_test.squeeze() == 1]
  break
print(malicious_samples.shape)

torch.Size([15, 10000])


In [52]:
benign_samples = []
for x_batch, y_batch in train_loader:
  benign_samples.append(x_batch[y_batch.squeeze() == 0])

ben_x = torch.cat(benign_samples, dim=0).to(device)
del benign_samples

print(ben_x.shape)

torch.Size([25417, 10000])


In [73]:
malicious_samples = malicious_samples[11:12]
malicious_samples.shape

torch.Size([1, 10000])

In [85]:
trials = 30
seed = 0
is_report_loss_diff = True

# Ensure trials do not exceed the length of ben_x
trials = min(trials, len(ben_x))

# Get the number of malicious samples
n_samples = len(malicious_samples)

if n_samples > 0:
    # Expand dimensions for efficient broadcasting
    malicious_samples_expanded = malicious_samples.unsqueeze(1).expand(-1, trials, -1)

    # Generate random indices for sampling from ben_x
    torch.manual_seed(seed)
    indices = torch.randperm(len(ben_x), device=ben_x.device)[:trials]
    trial_vectors_expanded = ben_x[indices].unsqueeze(0)

    # Perform the mimic attack
    pertbx = torch.clamp(malicious_samples_expanded + trial_vectors_expanded, min=0., max=1.)

    # Compute the loss and check if adversarial examples are successful
    loss, done = get_loss(pertbx.view(-1, pertbx.shape[-1]), torch.ones(n_samples * trials, 1, device=ben_x.device), model_AT_rFGSM_weightedLoss)

    # Add maximum loss to successful attacks to differentiate
    max_v = loss.max()
    print(max_v)
    loss[done] += max_v

    # Reshape the loss and done tensors
    loss = loss.view(n_samples, trials)
    done = done.view(n_samples, trials)
    print(done)
    # Report attack effectiveness if required
    if is_report_loss_diff:
        n_done = torch.any(done, dim=-1).sum()
        print(torch.any(done, dim=-1).sum())
        print(f"Mimicry*{trials}: Attack effectiveness {n_done / n_samples * 100:.3f}%.")


    # Get the index of the maximum loss for each sample
    _, indices = loss.max(dim=-1)
    adv_x = pertbx[torch.arange(n_samples), indices]

    del  done, malicious_samples_expanded, trial_vectors_expanded

tensor(2.8622, grad_fn=<MaxBackward1>)
tensor([[ True,  True, False,  True,  True,  True,  True,  True, False,  True,
          True,  True,  True, False,  True,  True,  True,  True,  True,  True,
          True,  True,  True, False,  True,  True, False,  True, False, False]])
tensor(1)
Mimicry*30: Attack effectiveness 100.000%.


In [82]:
outputs = model_AT_rFGSM_weightedLoss(adv_x)
outputs

tensor([[ 1.4397, -1.3637]], grad_fn=<AddmmBackward0>)

In [84]:
loss

tensor([[3.7591, 4.3977, 0.3276, 4.8397, 3.6425, 4.9612, 3.7863, 5.7244, 0.1577,
         4.2006, 4.9330, 4.2783, 4.1612, 0.1679, 5.3477, 5.3477, 5.1110, 4.6716,
         4.7087, 3.7372, 5.3477, 5.3477, 5.1408, 0.4178, 4.0100, 4.0743, 0.0556,
         4.4061, 0.2680, 0.1240]], grad_fn=<ViewBackward0>)

In [66]:
outputs

tensor([[-9.2042,  8.5523]], grad_fn=<AddmmBackward0>)

In [42]:
malicious_samples.sum()

tensor(539.)

In [ ]:
attack_params = {'step_lengths':{"l1": 1.0, "l2": 0.05, "linf": 0.001}, "steps":500}
adv_predict(test_loader, model_AT_rFGSM, StepwiseMax_onestep2, device, **attack_params)

step-wise max: attack effectiveness 0.000%.
step-wise max: attack effectiveness 10.000%.


KeyboardInterrupt: 

In [ ]:
# Define different attacks with their parameters
attacks = [
    (dfgsm_k, {"k":100, "epsilon":0.02, 'is_report_loss_diff' : False}),
    (bga_k, {"k":100, 'is_report_loss_diff' : False}),
    (bca_k, {"k":100, 'is_report_loss_diff' : False}),
    (grosse_k, {"k":100, 'is_report_loss_diff' : False}),

    (pgd, {'k': 100, 'step_length': 1., 'norm': 'l1', 'is_report_loss_diff' : False}),
    (pgd, {'k': 200, 'step_length': 0.05, 'norm': 'l2', 'is_report_loss_diff' : False}),
    (pgd, {'k': 100, 'step_length': 0.02, 'norm': 'linf', 'is_report_loss_diff' : False}),
    (StepwiseMax_onestep2, {'step_lengths':{"l1": 1.0, "l2": 0.05, "linf": 0.0008}, "steps":650}),

    # Add more attacks as needed
]

# Iterate over each attack and its parameters
for attack_func, attack_params in attacks:
    print(f"Running attack: {attack_func.__name__} with parameters: {attack_params}")
    adv_predict(test_loader, model_AT_rFGSM, attack_func, device, **attack_params)
    print()  # Print an empty line for separation


Running attack: dfgsm_k with parameters: {'k': 50, 'epsilon': 0.02, 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 83.43%.

Running attack: bga_k with parameters: {'k': 25, 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 91.57%.

Running attack: bca_k with parameters: {'k': 25, 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 87.93%.

Running attack: grosse_k with parameters: {'k': 25, 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 87.93%.

Running attack: pgd with parameters: {'k': 100, 'step_length': 1.0, 'norm': 'l1', 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 87.24%.

Running attack: pgd with parameters: {'k': 200, 'step_length': 0.05, 'norm': 'l2', 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 91.67%.

Running att